# Play a Skyjo game

In [5]:
# Imports

import importlib
import environment
import agents
import numpy as np

importlib.reload(environment)
importlib.reload(agents)

<module 'agents' from '/Users/paulsalquebre/Documents/SkyjoRL/agents.py'>

In [2]:
# A function to print the current board state

def print_board(state):
    """Print the board as described by the state"""
    player_turn = state["player_turn"]
    players_deck_i_values = state["player_deck_i_values"]
    players_deck_i_mask = state["player_deck_i_mask"]
    visible_card_value = state["visible_card_value"]
    done = state["done"]
    n_players = len(players_deck_i_values)

    if done:
        print("The game has ended!")
        for p in range(n_players):
            print(f"Player {p} has a score of {np.sum(players_deck_i_values[p])}")
    else:
        # Print player turn
        if player_turn[1] == 0:
            #print(f"Player {player_turn[0]} needs to draw a card.")
            print(f"Visible card: {visible_card_value}\n")
        else:
            #print(f"Player {player_turn[0]} has drawn the card {visible_card_value} from the draw pile or the visible card.\n")
            print(f"Card drawn: {visible_card_value}\n")

    # Print each player's deck
    for p in range(n_players):
        player_deck_values = players_deck_i_values[p]
        player_deck_mask = players_deck_i_mask[p]
        print(f"----- PLAYER {p} -----")
        for i in range(player_deck_values.shape[0]):
            row = "|"
            for j in range(player_deck_values.shape[1]):
                if player_deck_mask[i,j] == 1:
                    row += f"  {player_deck_values[i,j]:02}"
                else:
                    row += "  **"
            row += "  |"
            print(row)
        print("--------------------")
        print("")

In [ ]:
# Play a game

def play_game_render(env, players):
    observation = env.reset()
    done = False

    while not done:
        player_id = observation["player_turn"][0]
        player = players[player_id]
        print_board(env.get_state())
        action = player.get_action(observation)
        if action[0] == 0:
            print(f"The player {player_id} draws from the pile")
        else:
            print(f"The player {player_id} draws the visible card")
        observation, _, _ = env.step(action)
        #print_board(env.get_state())

        action = player.get_action(observation)
        print(f"The player {player_id} has drawn the card: {observation["visible_card"]}")
        if action[1] == 1:
            print(f"The player {player_id} use the drawn card and put it in the position {action[2]},{action[3]}")
        else:
            print(f"The player {player_id} discards the drawn card and uncover the card in the position {action[2]},{action[3]}")
        observation, reward, done = env.step(action)
        print("\n##########################################\n")

    print_board(env.get_state())


def play_game(env, players):
    num_turns = 0
    n_players = len(players)
    assert n_players == env.n_players
    observation = env.reset()
    done = False

    while not done:
        player_id = observation["player_turn"][0]
        player = players[player_id]

        if player_id == 0:
            num_turns += 1

        action = player.get_action(observation)
        observation, _, _ = env.step(action)

        action = player.get_action(observation)
        observation, reward, done = env.step(action)

    scores = np.zeros(n_players)
    for p in range(n_players):
        scores[p] = np.sum(env.players_deck_i_values[p])
    
    return scores, num_turns


In [54]:
# A singleplayer game, with the baseline agent

n_players = 4
env = environment.SkyjoEnv(n_players=n_players)

players = []
for i in range(n_players):
    players.append(agents.BaselineAgent(i))

In [55]:
play_game_render(env, players)

Visible card: 8

----- PLAYER 0 -----
|  01  02  **  **  |
|  **  **  **  **  |
|  **  **  **  **  |
--------------------

----- PLAYER 1 -----
|  08  11  **  **  |
|  **  **  **  **  |
|  **  **  **  **  |
--------------------

----- PLAYER 2 -----
|  11  08  **  **  |
|  **  **  **  **  |
|  **  **  **  **  |
--------------------

----- PLAYER 3 -----
|  07  09  **  **  |
|  **  **  **  **  |
|  **  **  **  **  |
--------------------

The player 0 draws from the pile
The player 0 has drawn the card: 0
The player 0 use the drawn card and put it in the position 1,1

##########################################

Visible card: 12

----- PLAYER 0 -----
|  01  02  **  **  |
|  **  00  **  **  |
|  **  **  **  **  |
--------------------

----- PLAYER 1 -----
|  08  11  **  **  |
|  **  **  **  **  |
|  **  **  **  **  |
--------------------

----- PLAYER 2 -----
|  11  08  **  **  |
|  **  **  **  **  |
|  **  **  **  **  |
--------------------

----- PLAYER 3 -----
|  07  09  **  **  |
|  **

In [63]:
scores = play_game(env, players)
print(scores)

(array([30., 15., 33., 20.]), 11)
